# IMPORTING ALL RELEVANT LIBRARIES

In [ ]:
# ------- [Import all relevant libraries] -------

# Utilities
import warnings
warnings.filterwarnings('ignore')

# Usual Suspects
import numpy as np           # Mathematical operations
import pandas as pd          # Data manipulation

# Visualization
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')
import seaborn as sns

# String manipulation
import re

# Pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# ML
# Models
import xgboost as xgb
import lightgbm as lgb

import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, ElasticNet, ElasticNetCV, Ridge

# ML Model Evaluation
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Time Series
!pip install optuna
import optuna

from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

!pip install catboost
from catboost import CatBoostRegressor

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge

# Display settings
pd.set_option('display.max_colwidth', None)
from IPython.display import display

# LOADING THE DATA

For this modeling section, we will use the data that has no feature engineering so we can focus on creating better features suited for modeling as opposed to EDA.

Once loaded, we will do basic IDE(Initial Data Exploration) to make sure the data is consistent from when we saved it.

In [ ]:
data = pd.read_csv("../Clean Data/data_no_feature_engineering.csv")

# Create a copy
df = data.copy(deep=True)

# Check the shape
print(f"The dataset contains {df.shape[0]} rows and {df.shape[1]} columns.")

# Check column names
print("\nColumn names:")
display(df.columns)

# Check metadata
print('\n'+"--"*50+'\n')
display(df.info())

# Check for duplicates and remove them
print('\n'+"--"*50+'\n')
print("Duplicates:", df.duplicated().sum())

# Drop duplicates
df.drop_duplicates(inplace=True)
print("Duplicates after removal:", df.duplicated().sum())

# Check for nulls
print('\n'+"--"*50+'\n')
print("Missing Values:")
display(df.isna().sum())

In [ ]:
# # Check for outliers
# plt.figure(figsize=(7, 4))
# sns.boxplot(data=df, palette="Set2", linewidth=1.2)
# plt.title("Outlier Summary")
# plt.xticks(rotation=0, ha="center")
# plt.tight_layout()
# plt.show()

# WRANGLING, BASIC EDA AND FEATURE ENGINEERING

We remove redundant columns and do quick EDA to understand how best to handle feature engineering for our goals.

In [ ]:
# Columns to drop
cols_to_drop = [
    'country_name',
    'transaction_date',
    'current_dollar_amount',
    'implementing_partner_name',
    'managing_subagency_or_bureau_name',
    'implementing_partner_category_name',
    'foreign_assistance_objective_name'
]

# Drop
df = df.drop(columns=cols_to_drop)
print("Remaining columns:")
display(df.columns)

In [ ]:
df['is_refund'] = (df['constant_dollar_amount'] < 0).astype(int)

# Convert all dollar amounts to positive magnitudes
df['constant_dollar_amount'] = df['constant_dollar_amount'].abs()

print("Negative values remaining:", (df['constant_dollar_amount'] < 0).sum())
print("Refund transactions:", df['is_refund'].sum())

In [ ]:
# --- Inspect managing agency counts ---
agency_counts = df['managing_agency_name'].value_counts()
display(agency_counts)

# --- Plot distribution ---
plt.figure(figsize=(12,7.5))
sns.barplot(x=agency_counts.index, y=agency_counts.values, palette="magma")
plt.xticks(rotation=90)
plt.title("Distribution of Projects by Managing Agency")
plt.ylabel("Number of Projects")
plt.xlabel("Managing Agency")
plt.tight_layout()
plt.show()

In [ ]:
# Inspect funding agency name
agency_counts = df['funding_agency_name'].value_counts()
display(agency_counts)

# --- Plot distribution ---
plt.figure(figsize=(10,7.5))
sns.barplot(x=agency_counts.index, y=agency_counts.values, palette="viridis")
plt.xticks(rotation=90)
plt.title("Distribution of Funding Agencies")
plt.ylabel("Number of Projects")
plt.xlabel("Funding Agency")
plt.tight_layout()
plt.show()

In [ ]:
# inspect us category name
df['us_category_name'].value_counts()

In [ ]:
# Inspect us_sector_name
df['us_sector_name'].value_counts()

In [ ]:
sector_mapping = {
    # --- Health ---
    'HIV/AIDS': 'Health',
    'Malaria': 'Health',
    'Tuberculosis': 'Health',
    'Maternal and Child Health': 'Health',
    'Family Planning and Reproductive Health': 'Health',
    'Health - General': 'Health',
    'Other Public Health Threats': 'Health',
    'Pandemic Influenza and Other Emerging Threats (PIOET)': 'Health',
    'Nutrition': 'Health',

    # --- Education ---
    'Basic Education': 'Education',
    'Higher Education': 'Education',
    'Education and Social Services - General': 'Education',

    # --- Governance & Human Rights ---
    'Rule of Law and Human Rights': 'Governance & Human Rights',
    'Good Governance': 'Governance & Human Rights',
    'Democracy, Human Rights, and Governance - General': 'Governance & Human Rights',
    'Civil Society': 'Governance & Human Rights',
    'Political Competition and Consensus-Building': 'Governance & Human Rights',
    'Monitoring and Evaluation': 'Governance & Human Rights',

    # --- Agriculture & Food Security ---
    'Agriculture': 'Agriculture & Food Security',

    # --- Economic Growth & Development ---
    'Economic Opportunity': 'Economic Growth & Development',
    'Economic Development - General': 'Economic Growth & Development',
    'Financial Sector': 'Economic Growth & Development',
    'Private Sector Competitiveness': 'Economic Growth & Development',
    'Trade and Investment': 'Economic Growth & Development',
    'Macroeconomic Foundation for Growth': 'Economic Growth & Development',
    'Policies, Regulations, and Systems': 'Economic Growth & Development',

    # --- Infrastructure & Environment ---
    'Infrastructure': 'Infrastructure & Environment',
    'Water Supply and Sanitation': 'Infrastructure & Environment',
    'Clean Productive Environment': 'Infrastructure & Environment',
    'Environment - General': 'Infrastructure & Environment',
    'Natural Resources and Biodiversity': 'Infrastructure & Environment',
    'Manufacturing': 'Infrastructure & Environment',
    'Mining and Natural Resources': 'Infrastructure & Environment',
    'Environment': 'Infrastructure & Environment',

    # --- Peace & Security ---
    'Stabilization Operations and Security Sector Reform': 'Peace & Security',
    'Counter-Terrorism': 'Peace & Security',
    'Counter-Narcotics': 'Peace & Security',
    'Conflict Mitigation and Reconciliation': 'Peace & Security',
    'Peace and Security - General': 'Peace & Security',
    'Transnational Crime': 'Peace & Security',
    'Combating Weapons of Mass Destruction (WMD)': 'Peace & Security',

    # --- Humanitarian & Social Protection ---
    'Protection, Assistance and Solutions': 'Humanitarian & Social Protection',
    'Disaster Readiness': 'Humanitarian & Social Protection',
    'Migration Management': 'Humanitarian & Social Protection',
    'Humanitarian Assistance - General': 'Humanitarian & Social Protection',
    'Social Services': 'Humanitarian & Social Protection',
    'Social Assistance': 'Humanitarian & Social Protection',

    # --- Governance & Administration ---
    'Direct Administrative Costs': 'Governance & Administration',
    'Multi-sector - Unspecified': 'Governance & Administration',
    'International Contributions': 'Governance & Administration',
    'Labor Policies and Markets': 'Governance & Administration',
}

# Apply mapping
df['sector'] = df['us_sector_name'].map(sector_mapping)
df.drop(columns='us_sector_name', inplace=True)

# --- Check distribution ---
sector_counts = df['sector'].value_counts(dropna=False)
display(sector_counts)

# --- Plot distribution ---
plt.figure(figsize=(10,6))
sns.barplot(x=sector_counts.index, y=sector_counts.values, palette="coolwarm")
plt.xticks(rotation=90, ha='center')
plt.title("Distribution of Projects by Sector")
plt.ylabel("Number of Projects")
plt.xlabel("Sector")
plt.tight_layout()
plt.show()

In [ ]:
# --- Count projects per fiscal year ---
fy_counts = df['fiscal_year'].value_counts().sort_index()  # chronological order
print(fy_counts)

# --- Line plot ---
plt.figure(figsize=(10,4))
sns.lineplot(x=fy_counts.index, y=fy_counts.values, marker='o', color='teal')
plt.title("Trend of Projects by Fiscal Year")
plt.xlabel("Fiscal Year")
plt.ylabel("Number of Projects")
plt.grid(True)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Drop us_category_name
df.drop(columns='us_category_name', inplace=True)

In [ ]:
# Sort for time series
df = df.sort_values(['managing_agency_name', 'sector', 'fiscal_year']).reset_index(drop=True)
df.head()

# FEATURE ENGINEERING

In [ ]:
# --- FEATURE ENGINEERING ---
group_cols = ['managing_agency_name', 'sector']
target_col = 'constant_dollar_amount'

# Define lag periods and rolling windows
lags = [1, 2]
rolling_windows = [3]

# --- Lag features ---
for lag in lags:
    df[f'lag_{lag}'] = df.groupby(group_cols)[target_col].shift(lag)

# --- Rolling mean and std ---
for window in rolling_windows:
    df[f'rolling_mean_{window}yr'] = df.groupby(group_cols)[target_col].transform(
        lambda x: x.shift(1).rolling(window, min_periods=1).mean()
    )
    df[f'rolling_std_{window}yr'] = df.groupby(group_cols)[target_col].transform(
        lambda x: x.shift(1).rolling(window, min_periods=1).std()
    )

# --- Growth rate ---
df['funding_growth_rate'] = df.groupby(group_cols)[target_col].pct_change()

# --- Fill NaNs with 0 ---
feat_cols = [f'lag_{lag}' for lag in lags] + \
            [f'rolling_mean_{w}yr' for w in rolling_windows] + \
            [f'rolling_std_{w}yr' for w in rolling_windows] + \
            ['funding_growth_rate']

df[feat_cols] = df[feat_cols].fillna(0)

# Preview
df.head(10)

# MODELING

## VANILLA MODELING

In [ ]:
# --- ML PIPELINES FOR RANDOM FOREST, XGBOOST, LIGHTGBM AND CATBOOST ---

# --- 1. Separate features and target ---
X = df.drop('constant_dollar_amount', axis=1)
y = np.log1p(df['constant_dollar_amount'])  

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical and numeric features
cat_cols = x_train.select_dtypes(include=['object']).columns.tolist()
num_cols = [c for c in x_train.columns if c not in cat_cols]

# --- 2. Column transformer for preprocessing ---
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), cat_cols),
        ('num', 'passthrough', num_cols)
    ]
)

# --- 3. Define pipelines for each model ---
pipelines = {
    'RandomForest': Pipeline([
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor(random_state=42))
    ]),
    'XGBoost': Pipeline([
        ('preprocessor', preprocessor),
        ('model', xgb.XGBRegressor(random_state=42, objective='reg:squarederror'))
    ]),
    'LightGBM': Pipeline([
        ('preprocessor', preprocessor),
        ('model', lgb.LGBMRegressor(random_state=42))
    ]),
    'Catboost': Pipeline([
        ('preprocessor', preprocessor),
        ('model', CatBoostRegressor(random_state=42, verbose=0))
    ])
}

# --- 4. Train and evaluate ---
for name, pipe in pipelines.items():
    pipe.fit(x_train, y_train)
    y_pred_log = pipe.predict(x_test)
    y_pred = np.expm1(y_pred_log)  
    y_true = np.expm1(y_test)

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    print(f"--- {name} ---")
    print(f"MAE: {mae:,.2f}")
    print(f"RMSE: {rmse:,.2f}")
    print(f"R²: {r2:.4f}\n")


## ENSEMBLE STACKING OF BEST MODELS

In [ ]:
# --- Base Models ---
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
cat_model = CatBoostRegressor(random_state=42, verbose=0)

# --- 5. Meta Model ---
meta_model = Ridge()

# --- 6. Stacking Regressor ---
stack_model = StackingRegressor(
    estimators=[
        ('xgb', xgb_model),
        ('cat', cat_model)
    ],
    final_estimator=meta_model,
    n_jobs=-1
)

# --- 7. Pipeline ---
stack_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', stack_model)
])

# --- 8. Fit and Evaluate ---
print("Training vanilla stacking model...\n")
stack_pipe.fit(x_train, y_train)

# Predictions
y_pred_log = stack_pipe.predict(x_test)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

# Metrics
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)

# --- 9. Report ---
print("=== VANILLA STACKING ENSEMBLE (XGB + CatBoost) ===")
print(f"MAE: {mae:,.2f}")
print(f"RMSE: {rmse:,.2f}")
print(f"R²: {r2:.4f}")


In [ ]:
# === VANILLA STACKING REGRESSION (XGB +  + RIDGE) ===

# --- 1. Split features and target ---
X = df.drop('constant_dollar_amount', axis=1)
y = np.log1p(df['constant_dollar_amount'])  # Log-transform for stability

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# --- 2. Identify categorical and numeric columns ---
cat_cols = x_train.select_dtypes(include=['object']).columns.tolist()
num_cols = [c for c in x_train.columns if c not in cat_cols]

# --- 3. Preprocessor ---
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False), cat_cols),
        ('num', StandardScaler(), num_cols)
    ]
)

# --- 4. Base Models ---
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
lgb_model = lgb.LGBMRegressor(random_state=42)

# --- 5. Meta Model ---
meta_model = Ridge()

# --- 6. Stacking Regressor ---
stack_model = StackingRegressor(
    estimators=[
        ('xgb', xgb_model),
        ('lgb', lgb_model)
    ],
    final_estimator=meta_model,
    n_jobs=-1
)

# --- 7. Pipeline ---
stack_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', stack_model)
])

# --- 8. Fit and Evaluate ---
print("Training vanilla stacking model...\n")
stack_pipe.fit(x_train, y_train)

# Predictions
y_pred_log = stack_pipe.predict(x_test)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

# Metrics
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)

# --- 9. Report ---
print("=== VANILLA STACKING ENSEMBLE ===")
print(f"MAE: {mae:,.2f}")
print(f"RMSE: {rmse:,.2f}")
print(f"R²: {r2:.4f}")

## TUBED STACKING

In [ ]:
# === TUNED STACKING REGRESSION (XGB + LGB + RIDGE with RandomizedSearchCV) ===

# --- 1. Split features and target ---
X = df.drop('constant_dollar_amount', axis=1)
y = np.log1p(df['constant_dollar_amount'])

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# --- 2. Identify categorical and numeric columns ---
cat_cols = x_train.select_dtypes(include=['object']).columns.tolist()
num_cols = [c for c in x_train.columns if c not in cat_cols]

# --- 3. Preprocessor ---
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False), cat_cols),
        ('num', StandardScaler(), num_cols)
    ]
)

# --- 4. Base Models ---
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
lgb_model = lgb.LGBMRegressor(random_state=42)
meta_model = Ridge()

stack_model = StackingRegressor(
    estimators=[
        ('xgb', xgb_model),
        ('lgb', lgb_model)
    ],
    final_estimator=meta_model,
    n_jobs=-1
)

stack_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', stack_model)
])

# --- 5. Parameter Grid for RandomizedSearchCV ---
param_grid = {
    # XGBoost
    'model__xgb__n_estimators': [100, 200, 300],
    'model__xgb__max_depth': [3, 4, 5, 6, 8],
    'model__xgb__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'model__xgb__subsample': [0.6, 0.8, 1.0],
    'model__xgb__colsample_bytree': [0.6, 0.8, 1.0],

    # LightGBM
    'model__lgb__n_estimators': [100, 200, 300],
    'model__lgb__num_leaves': [20, 31, 50, 70],
    'model__lgb__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'model__lgb__subsample': [0.6, 0.8, 1.0],
    'model__lgb__colsample_bytree': [0.6, 0.8, 1.0],

    # Ridge meta-model
    'model__final_estimator__alpha': np.logspace(-3, 2, 20)
}

# --- 6. Randomized Search ---
random_search = RandomizedSearchCV(
    stack_pipe,
    param_distributions=param_grid,
    n_iter=25,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=2,
    random_state=42
)

print("Running Randomized Search for Stacking Ensemble...\n")
random_search.fit(x_train, y_train)

# --- 7. Evaluation ---
best_model = random_search.best_estimator_

y_pred_log = best_model.predict(x_test)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)

print("\n=== TUNED STACKING ENSEMBLE RESULTS ===")
print(f"Best Params: {random_search.best_params_}")
print(f"MAE: {mae:,.2f}")
print(f"RMSE: {rmse:,.2f}")
print(f"R²: {r2:.4f}")


In [ ]:
# === TIME SERIES CROSS-VALIDATION + RANDOMIZED SEARCH FOR XGB ===

# ---
# We use time series cross validation for evaluating time series data
# ---

# --- 1. TimeSeriesSplit setup ---
tscv = TimeSeriesSplit(n_splits=8)

# --- 2. Model parameter grids ---

# XGBoost
xgb_params = {
    'model__n_estimators': [100, 300],
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__max_depth': [4, 6, 8],
    'model__subsample': [0.7, 0.9, 1.0],
    'model__colsample_bytree': [0.7, 0.9, 1.0],
    'model__reg_lambda': [1, 2, 5]
}

# --- 3. Model pipelines ---
pipelines = {
    'XGBoost': Pipeline([
        ('preprocessor', preprocessor),
        ('model', xgb.XGBRegressor(random_state=42, objective='reg:squarederror'))
    ])
}

# --- 4. Randomized Search ---
param_grids = {'XGBoost': xgb_params}

best_models = {}
for name, pipe in pipelines.items():
    print(f"\nTuning {name} with TimeSeriesSplit...\n")

    search = RandomizedSearchCV(
        estimator=pipe,
        param_distributions=param_grids[name],
        n_iter=20,               # test 20 random combinations
        cv=tscv,
        scoring='r2',
        verbose=2,
        n_jobs=-1,
        random_state=42
    )

    search.fit(X, y)
    best_models[name] = search.best_estimator_

    print(f"Best R² (CV): {search.best_score_:.4f}")
    print(f"Best Params: {search.best_params_}\n")

# --- 5. Evaluate on final holdout (last few years, e.g. 2020–2025) ---
holdout = df[df['fiscal_year'] >= 2020]
train = df[df['fiscal_year'] < 2020]

X_train = train.drop('constant_dollar_amount', axis=1)
y_train = np.log1p(train['constant_dollar_amount'])
X_test = holdout.drop('constant_dollar_amount', axis=1)
y_test = np.log1p(holdout['constant_dollar_amount'])

for name, model in best_models.items():
    y_pred_log = model.predict(X_test)
    y_pred = np.expm1(y_pred_log)
    y_true = np.expm1(y_test)

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    print(f"--- {name} (Final Holdout Evaluation) ---")
    print(f"MAE: {mae:,.2f}")
    print(f"RMSE: {rmse:,.2f}")
    print(f"R²: {r2:.4f}\n")


In [ ]:
# === OPTUNA OPTIMIZATION FOR XGBOOST WITH TIME SERIES SPLIT ===

# --- 1. TimeSeriesSplit setup ---
tscv = TimeSeriesSplit(n_splits=8)

# --- 2. Define Optuna objective ---
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 10.0, log=True),
        'random_state': 42,
        'objective': 'reg:squarederror',
        'n_jobs': -1
    }

    pipe = Pipeline([
        ('preprocessor', preprocessor),
        ('model', xgb.XGBRegressor(**params))
    ])

    scores = cross_val_score(pipe, X, y, cv=tscv, scoring='r2', n_jobs=-1)
    return np.mean(scores)

# --- 3. Run Optuna study ---
print("=== OPTIMIZING XGBOOST (Optuna) ===")
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30, show_progress_bar=True)

print(f"\nBest R² (CV): {study.best_value:.4f}")
print("Best Params:", study.best_params)

# --- 4. Fit best model ---
best_xgb = Pipeline([
    ('preprocessor', preprocessor),
    ('model', xgb.XGBRegressor(**study.best_params))
]).fit(X, y)

# --- 5. Final holdout evaluation (2020–2025) ---
holdout = df[df['fiscal_year'] >= 2020]
train = df[df['fiscal_year'] < 2020]

X_train = train.drop('constant_dollar_amount', axis=1)
y_train = np.log1p(train['constant_dollar_amount'])
X_test = holdout.drop('constant_dollar_amount', axis=1)
y_test = np.log1p(holdout['constant_dollar_amount'])

# Predictions
y_pred_log = best_xgb.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

# Metrics
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)

print(f"\n--- XGBoost (Final Holdout Evaluation) ---")
print(f"MAE: {mae:,.2f}")
print(f"RMSE: {rmse:,.2f}")
print(f"R²: {r2:.4f}")

In [ ]:
# ---
# Save our best model- XGBoost with RandomizedSearchCV optimization
# ---

import joblib

joblib.dump(best_models['XGBoost'], 'xgb_best_pipeline.pkl')

print("XGBoost pipeline saved successfully!")
